In [ ]:
import pandas as pd
import datetime, pytz
import random

### Read users

In [ ]:
users = pd.read_excel("../../data/users_processed.xlsx")

In [ ]:
## Shuffle users 
users = users.sample(frac=1).reset_index(drop=True)

### Read workflow def

In [ ]:
teams_df = pd.read_excel("../../data/workflow.xlsx", sheet_name="teams")
teams = teams_df['Team'].tolist()

steps_df = pd.read_excel("../../data/workflow.xlsx", sheet_name="steps")
step_names = steps_df['Step'].tolist()

users_df = pd.read_excel("../../data/users_processed.xlsx")


In [ ]:
tz_list = users_df['tz'].unique()


In [ ]:
tz_list

In [ ]:
def get_rows_by_val(df, col, val, reset_index=True):
    temp_df = df.loc[df[col] == val]
    if reset_index:
        temp_df.reset_index(drop=True, inplace=True)
    return temp_df    

In [ ]:
def get_users_by_team(user_df, team):
    temp_df = get_rows_by_val(user_df, 'team', team)
    return temp_df

In [ ]:
team_col_idx = 1
print("Reading teams")
for s in step_names:
    row_df = steps_df.loc[steps_df['Step'] == s]
    team_name = row_df.iat[0,1]
    tmp_df = get_users_by_team(users_df,team_name)
    user_count = len(tmp_df.index)
    print("step : " + s +" team : "+team_name + " , #users : "+str(user_count))



### Generate tasks for step 1 

In [ ]:
step = step_names[0]
task_count = 5000
for i in range(task_count):
    task_type = step
    task_tz = 
    task = {'task_type':step}
    